In [ ]:
! pip install talib-binary
! gdown 17OYeD6dlfwX28FNoUpqXkLVpzjUtTCRW
! unzip /content/ThaiDepression.zip
! pip install -qqq greykite
! pip install --upgrade pandas
! python -m pip uninstall matplotlib
! pip install matplotlib==3.1.3
! pip install tsai
! pip install sktime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 5.2 MB/s 
Downloading...
From: https://drive.google.com/uc?id=17OYeD6dlfwX28FNoUpqXkLVpzjUtTCRW
To: /content/ThaiDepression.zip
100% 8.78M/8.78M [00:00<00:00, 65.7MB/s]
Archive:  /content/ThaiDepression.zip
   creating: ThaiDepression/2016/
  inflating: ThaiDepression/2016/ahb-2016-10.csv  
  inflating: ThaiDepression/2016/ahb-2016-10.pdf  
  inflating: ThaiDepression/2016/ahb-2016-11.csv  
  inflating: ThaiDepression/2016/ahb-2016-11.pdf  
  inflating: ThaiDepression/2016/ahb-2016-12.csv  
  inflating: ThaiDepression/2016/ahb-2016-12.pdf  
   creating: ThaiDepression/2017/
  inflating: ThaiDepression/2017/ahb-2017-01.csv  
  inflating: ThaiDepression/2017/ahb-2017-01.pdf  
  inflating: ThaiDepression/2017/ahb-2017-02.csv  
  inflating: ThaiDepression/2017/ahb-2017-02.pdf  
  inflating: ThaiDepression/2017/ahb-2017-03.csv  
  inflating: ThaiD

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 241 kB 5.2 MB/s 
     |████████████████████████████████| 750.6 MB 11 kB/s 
     |████████████████████████████████| 2.5 MB 44.5 MB/s 
     |████████████████████████████████| 19.1 MB 1.2 MB/s 
     |████████████████████████████████| 19.1 MB 823 kB/s 
     |████████████████████████████████| 21.0 MB 7.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import talib
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster 
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
import plotly
import warnings
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import  plot_pacf
from statsmodels.tsa.stattools import pacf
import itertools
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from tsai.all import *
from tsai.inference import load_learner
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
from pmdarima import arima

In [ ]:
path = "/content/ThaiDepression/"
year_list = os.listdir(path)
year_list.sort()

df_list = []
name_list = []
 
for i in range(len(year_list)):
    file_list = os.listdir(path+year_list[i])
    file_list.sort()
    for f in file_list:
        if '.csv' in f:
            # print(f)
            name_list.append(f[4:11])
            df_list.append(pd.read_csv(path+year_list[i]+'/'+f, header=None, index_col=False))


finished_process = []
for i in range(len(df_list[0])):
    finished_process.append(pd.DataFrame(df_list[0].iloc[i,:]).T.reset_index(drop=True))
    finished_process[i]['y-m'] = name_list[0]


i=1
while(i != len(df_list)):
    # print('i=' + str(i))
    if len(df_list[i]) == 89:
        insert = { 0 : np.nan, 1 : np.nan, 2 : np.nan, 3 : np.nan, 4 : np.nan, 5 : np.nan, 6 :np.nan, 7 : np.nan, 8 : np.nan, 9 : np.nan, 'y-m' : name_list[i]}
        finished_process[89] = pd.concat([finished_process[89], pd.DataFrame(insert, index=[0])], ignore_index = True)
    for j in range(len(df_list[i])):
        # print('j=' + str(j))
        if len(df_list[i].columns) == 6 :
            insert = { 0 : df_list[i].iloc[j,0], 1 : df_list[i].iloc[j,1], 2 : np.nan, 3 : df_list[i].iloc[j,2], 4 : df_list[i].iloc[j,3], 5 : np.nan, 6 :np.nan, 7 : df_list[i].iloc[j,4], 8 : np.nan, 9 : df_list[i].iloc[j,5], 'y-m' : name_list[i]}
            finished_process[j] = pd.concat([finished_process[j], pd.DataFrame(insert, index=[0])], ignore_index = True)
        else:
            finished_process[j] = pd.concat([finished_process[j], pd.DataFrame(df_list[i].iloc[j,:]).T.reset_index(drop=True)], ignore_index = True)
            finished_process[j]['y-m'][i] = name_list[i]
    i=i+1


for i in range(len(finished_process)):
    finished_process[i].index = finished_process[i]['y-m'].values
    finished_process[i].drop(['y-m'], axis=1, inplace=True)
    # finished_process[i].index = pd.to_datetime(finished_process[i].index, format='%Y-%m')
    finished_process[i] = finished_process[i].replace(',','', regex=True)  
    finished_process[i] = finished_process[i].apply(pd.to_numeric)

    # pct_change(1) = (value on this index - value on previous index) / value on previous index

    finished_process[i]['0_pct_change'] = finished_process[i][0].pct_change(1)
    finished_process[i]['1_pct_change'] = finished_process[i][1].pct_change(1)
    finished_process[i]['2_pct_change'] = finished_process[i][2].pct_change(1)
    finished_process[i]['3_pct_change'] = finished_process[i][3].pct_change(1)
    finished_process[i]['4_pct_change'] = finished_process[i][4].pct_change(1)
    finished_process[i]['5_pct_change'] = finished_process[i][5].pct_change(1)
    finished_process[i]['6_pct_change'] = finished_process[i][6].pct_change(1)
    finished_process[i]['7_pct_change'] = finished_process[i][7].pct_change(1)
    finished_process[i]['8_pct_change'] = finished_process[i][8].pct_change(1)
    finished_process[i]['9_pct_change'] = finished_process[i][9].pct_change(1)
    # finished_process[i].loc[finished_process[i][3] == np.nan, '3_pct_change'] = np.nan
    # finished_process[i]['3_pct_change'] = finished_process[i][3].apply(lambda x: np.nan if x == np.nan else x)

In [ ]:
# 0 9 15 21 30 39 48 53 61 66 72 80 88 89(Bangkok)
y_row = [0, 9, 15, 21, 30, 39, 48, 53, 61, 66, 72, 80, 88, 89]
all_y = []
for i in range(len(y_row)):
     all_y.append(finished_process[y_row[i]][3])

all_y[-1].index = np.arange(len(all_y[-1].index))
all_y.append(all_y[-1].interpolate(method='polynomial', order=5))
first_interpolate = all_yww[-1].interpolate(method='polynomial', order=5)
first_interpolate.index = all_y[-3].index
all_y[-2].index = all_y[-3].index


warnings.filterwarnings("ignore")
forecast = pd.DataFrame(list(pd.to_datetime(all_y[-3].index, format='%Y-%m')), columns=['ts'])
forecast['y'] = list(all_y[-1])
# Specifies dataset information
metadata = MetadataParam(
     time_col="ts",  # name of the time column
     value_col="y",  # name of the value column
     freq="MS"  #"MS" for Montly at start date, "H" for hourly, "D" for daily, "W" for weekly, etc.
 )
forecaster = Forecaster()
result = forecaster.run_forecast_config(
     df=forecast,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE.name,
         forecast_horizon=100,  # forecasts 100 steps ahead
         coverage=0.95,  # 95% prediction intervals
         metadata_param=metadata
    )
)
warnings.filterwarnings("default")

all_y[-1] = pd.concat([all_y[-1].dropna(), result.forecast.df['forecast'][36:70]])
all_y[-1].index = all_y[-3].index
all_y[-1] = all_y[-1].astype(int)

all_y.append(all_y[0]-first_interpolate.fillna(0)+all_y[14])
all_y[-1]['2018-10'] = all_y[-1]['2018-10'] + first_interpolate['2018-10']

Fitting 1 folds for each of 1 candidates, totalling 1 fits


In [ ]:
AHB_data = []
AHB_list = [9, 15, 21, 30, 39, 48, 53, 61, 66, 72, 80, 88]
province_AHB_list = [
            [0, 1, 2, 3, 4, 5, 6, 7, 8],
            [0, 10, 11, 12, 13, 14],
            [0, 16, 17, 18, 19, 20],
            [0, 22, 23, 24, 25, 26, 27, 28, 29],
            [0, 31, 32, 33, 34, 35, 36, 37, 38],
            [0, 40, 41, 42, 43, 44, 45, 46, 47],
            [0, 49, 50, 51, 52],
            [0, 54, 55, 56, 57, 58, 59, 60],
            [0, 62, 63, 64, 65],
            [0, 67, 68, 69, 70, 71],
            [0, 73, 74, 75, 76, 77, 78, 79],
            [0, 81, 82, 83, 84, 85, 86, 87]
            ]
# AHB
for j in range(len(AHB_list)):
    AHB = pd.DataFrame(finished_process[AHB_list[j]][3].values, columns=['AHB_'+str(j+1)])
    for i in range(len(province_AHB_list[j])):
        if i == 0:
            continue
        AHB = pd.concat([AHB, finished_process[province_AHB_list[j][i]][3].reset_index(drop='index')], axis = 1)
    temp = [str(x) for x in province_AHB_list[j]]
    temp[0] = 'AHB_'+str(j+1)
    AHB.columns = temp
    AHB.index = finished_process[9][3].index
    AHB_data.append(AHB)

for j in range(len(AHB_list)):
    temp = AHB_data[j]['AHB_'+str(j+1)]
    AHB_data[j].drop(['AHB_'+str(j+1)],inplace=True, axis=1)
    AHB_data[j] = pd.concat([AHB_data[j], temp], axis=1)
    AHB_data[j] = pd.concat([AHB_data[j], all_y[15]], axis = 1)

province_AHB_list_name = [['Chiang_Rai', 'Nan', 'Phayao', 'Phrae', 'Chiang_Mai', 'Mae_Hong_Son', 'Lampang', 'Lamphun', 'AHB_1', 'Country_Level'],
                          ['Tak', 'Phitsanulok', 'Phetchabun', 'Sukhothai', 'Uttaradit', 'AHB_2', 'Country_Level'],
                          ['Chai_Nat', 'Kamphaeng_Phet', 'Phichit', 'Nakhon_Sawan', 'Uthai_Thani', 'AHB_3', 'Country_Level'],
                          ['Nonthaburi', 'Pathum_Thani', 'Phra_Nakhon Si_Ayutthaya', 'Saraburi', 'Lopburi', 'Sing_Buri', 'Ang_Thong', 'Nakhon_Nayok', 'AHB_4', 'Country_Level'],
                          ['Kanchanaburi', 'Nakhon_Pathom', 'Ratchaburi', 'Suphan_Buri', 'Prachuap_Khiri_Khan', 'Phetchaburi', 'Samut_Songkhram', 'Samut_Sakhon', 'AHB_5', 'Country_Level'],
                          ['Chachoengsao', 'Prachinburi', 'Sa_Kaeo', 'Samut_Prakan', 'Chanthaburi', 'Chonburi', 'Trat', 'Rayong', 'AHB_6', 'Country_Level'],
                          ['Kalasin', 'Khon_Kaen', 'Maha_Sarakham', 'Roi_Et', 'AHB_7', 'Country_Level'],
                          ['Bueng_Kan', 'Loei', 'Nong_Khai', 'Nong_Bua_Lamphu', 'Udon_Thani', 'Nakhon_Phanom', 'Sakon_Nakhon', 'AHB_8', 'Country_Level'],
                          ['Chaiyaphum', 'Nakhon_Ratchasima', 'Buriram', 'Surin', 'AHB_9', 'Country_Level'],
                          ['Mukdahan', 'Yasothon', 'Sisaket', 'Ubon_Ratchathani', 'Amnat_Charoen', 'AHB_10', 'Country_Level'],
                          ['Chumphon', 'Nakhon_Si_Thammarat', 'Surat_Thani', 'Krabi', 'Phang_Nga', 'Phuket', 'Ranong', 'AHB_11', 'Country_Level'],
                          ['Phatthalung', 'Trang', 'Narathiwat', 'Pattani', 'Yala', 'Songkhla', 'Satun', 'AHB_12', 'Country_Level']
                          ]

for j in range(len(AHB_list)):
     AHB_data[j].columns = province_AHB_list_name[j]
     AHB_data[j].to_csv('AHB_'+str(j+1)+'.csv')

AHB13 = pd.DataFrame(all_y[14], columns=['AHB_13'])
AHB13 = pd.concat([AHB13, AHB13], axis = 1)
AHB13 = pd.concat([AHB13, all_y[15]], axis = 1)
AHB13.columns = ['Bangkok', 'AHB_13', 'Country_Level']
AHB13.to_csv('AHB_13.csv')


AHB_1_col = AHB_data[0]['AHB_1']
AHB_2_col = AHB_data[1]['AHB_2']
AHB_3_col = AHB_data[2]['AHB_3']
AHB_4_col = AHB_data[3]['AHB_4']
AHB_5_col = AHB_data[4]['AHB_5']
AHB_6_col = AHB_data[5]['AHB_6']
AHB_7_col = AHB_data[6]['AHB_7']
AHB_8_col = AHB_data[7]['AHB_8']
AHB_9_col = AHB_data[8]['AHB_9']
AHB_10_col = AHB_data[9]['AHB_10']
AHB_11_col = AHB_data[10]['AHB_11']
AHB_12_col = AHB_data[11]['AHB_12']
AHB_13_col = AHB13['AHB_13']


AHB_country = pd.concat([AHB_data[0].drop(['Country_Level','AHB_1'],axis=1), AHB_data[1].drop(['Country_Level','AHB_2'],axis=1)], axis=1)
for j in range(len(AHB_list)):
    if j == 0 or j == 1:
        continue
    AHB_country = pd.concat([AHB_country, AHB_data[j].drop(['Country_Level','AHB_'+str(j+1)],axis=1)], axis=1)
AHB_country = pd.concat([AHB_country, AHB13.drop(['Country_Level','AHB_13'],axis=1)], axis=1)
AHB_country = pd.concat([AHB_country, AHB_1_col, AHB_2_col, AHB_3_col, AHB_4_col, AHB_5_col, AHB_6_col, AHB_7_col, AHB_8_col, AHB_9_col, AHB_10_col, AHB_11_col, AHB_12_col, AHB_13_col, AHB13['Country_Level']],axis=1)
AHB_country.to_csv('All_AHB.csv')


In [ ]:
AHB_country

,Chiang_Rai,Nan,Phayao,Phrae,Chiang_Mai,Mae_Hong_Son,Lampang,Lamphun,Tak,Phitsanulok,...,AHB_5,AHB_6,AHB_7,AHB_8,AHB_9,AHB_10,AHB_11,AHB_12,AHB_13,Country_Level
2016-10,10383,5315,4266,4057,15408,2002,9011,3214,2558,7875,...,43044,38746,54437,66020,71525,46691,39516,52629,23244,607299.0
2016-11,10383,5315,4265,4057,15406,2002,9011,3214,2558,7895,...,43691,38789,54776,66074,72897,47805,39587,52966,23251,611508.0
2016-12,10382,5315,4265,4057,15404,2002,9011,3214,2558,7895,...,43747,38787,55087,66392,73177,48152,39749,53279,23251,614080.0
2017-01,10382,5655,4265,4057,15393,2002,9011,3213,2558,7892,...,44651,40075,55661,66664,73739,48704,40062,53848,23243,620732.0
2017-02,10382,5655,4264,4057,15392,2002,9011,3213,2558,7946,...,45269,40108,56045,66809,77859,49247,40223,54214,23231,627688.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03,25223,9711,10788,7988,47292,6924,17444,12499,10165,18907,...,83252,84627,99574,102022,144393,101478,77972,92725,39613,1213727.0
2022-04,25456,9781,10932,8095,48020,7004,17606,12650,10318,19057,...,84282,85627,100517,102461,145690,102176,78767,93639,39933,1225461.0
2022-05,25619,9833,10977,8122,48155,7026,17703,12717,10320,19148,...,84534,85935,100726,103219,146809,102429,78995,94019,40963,1231355.0
2022-06,25836,9884,11069,8161,48616,7090,17783,12807,10323,19298,...,85008,86644,101269,103680,148170,102850,79596,94776,41324,1239786.0
